# Criação dos datasets considerando os cursos de TI do Brasil e da Bahia

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_style('whitegrid')

In [3]:
df_sup_aluno_tic = pd.read_csv('dataset/SUP_ALUNO_TIC_2015_2019.CSV', encoding='ISO-8859-1')
df_curso_tic = pd.read_csv('dataset/SUP_CURSO_TIC_2015_2019.CSV', usecols=['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO', 'NO_CURSO', 'CO_UF'], encoding='ISO-8859-1')
df_dados_ies = pd.read_csv('dataset/SUP_IES_2015_2019.CSV', encoding='ISO-8859-1', usecols=['NU_ANO_CENSO', 'CO_IES', 'SG_IES', 'CO_UF'])


In [4]:
df_sup_aluno_tic.head()

,NU_ANO_CENSO,CO_IES,TP_CATEGORIA_ADMINISTRATIVA,TP_ORGANIZACAO_ACADEMICA,CO_CURSO,CO_CURSO_POLO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,...,TP_MOBILIDADE_ACADEMICA,TP_MOBILIDADE_ACADEMICA_INTERN,CO_IES_DESTINO,CO_PAIS_DESTINO,IN_MATRICULA,IN_CONCLUINTE,IN_INGRESSO_TOTAL,IN_INGRESSO_VAGA_NOVA,IN_INGRESSO_PROCESSO_SELETIVO,NU_ANO_INGRESSO
0,2015,572,1,1,82799,NaN,4.0,1.0,1,1,...,NaN,NaN,NaN,NaN,1,0,0,0,NaN,2008
1,2015,572,1,1,82799,NaN,4.0,1.0,1,1,...,NaN,NaN,NaN,NaN,1,0,0,0,NaN,2011
2,2015,572,1,1,82799,NaN,4.0,1.0,1,1,...,NaN,NaN,NaN,NaN,1,0,0,0,NaN,2009
3,2015,572,1,1,82799,NaN,4.0,1.0,1,1,...,NaN,NaN,NaN,NaN,1,0,0,0,NaN,2009
4,2015,572,1,1,82799,NaN,4.0,1.0,1,1,...,NaN,NaN,NaN,NaN,1,0,0,0,NaN,2009


In [5]:
df_curso_tic = df_curso_tic.fillna(0)
df_dados_ies = df_dados_ies.fillna(0)
df_curso_tic = df_curso_tic.astype({'CO_UF': 'int32'})

df_dados_ies = df_dados_ies.replace({'CO_UF':{' CULTURA E TECNOLOGIA SAO FIDELIS LTDA - EPP': 33}})
df_dados_ies = df_dados_ies.astype({'CO_UF': 'int32'})

In [6]:
outer_merged = pd.merge(df_sup_aluno_tic, df_curso_tic, how="outer", on=['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO'])

In [7]:
outer_merged = pd.merge(outer_merged, df_dados_ies, how='outer', on=['NU_ANO_CENSO', 'CO_IES', 'CO_UF'])

## Filtros

Selecionando apenas os cursos da Bahia

In [8]:
curso_aluno_BA = outer_merged.loc[outer_merged.CO_UF == 29]
display(curso_aluno_BA.shape)

(63627, 108)

Selecionando apenas os cursos presenciais

In [9]:
display(curso_aluno_BA.TP_MODALIDADE_ENSINO.unique())
display(curso_aluno_BA.TP_MODALIDADE_ENSINO.value_counts())
curso_presencial = curso_aluno_BA.loc[curso_aluno_BA.TP_MODALIDADE_ENSINO == 1]
display(curso_presencial.shape)

array([ 1., nan])

1.0    63210
Name: TP_MODALIDADE_ENSINO, dtype: int64

(63210, 108)

Selecionando apenas os cursos de Universidades públicas

In [10]:
curso_presencial  = curso_presencial.loc[curso_presencial.TP_CATEGORIA_ADMINISTRATIVA.isin([1, 2, 3])]
curso_presencial.shape

(24456, 108)

selecionando apenas os alunos cursando

In [11]:
curso_presencial = curso_presencial.loc[curso_presencial['TP_SITUACAO'] == 2]
curso_presencial.groupby(['NO_CURSO']).size()

NO_CURSO
ANÁLISE E DESENVOLVIMENTO DE SISTEMAS    2671
CIÊNCIA DA COMPUTAÇÃO                    3898
CIÊNCIAS DA COMPUTAÇÃO                    533
COMPUTAÇÃO                               2649
ENGENHARIA DA COMPUTAÇÃO                 1199
ENGENHARIA DE COMPUTAÇÃO                 3101
JOGOS DIGITAIS                            112
REDES DE COMPUTADORES                      79
SISTEMA DE INFORMAÇÃO                     477
SISTEMAS DE INFORMAÇÃO                   3695
dtype: int64

In [12]:
curso_presencial.groupby(['CO_IES', 'SG_IES']).size()

CO_IES  SG_IES
24      UESC       972
40      UNEB      1467
578     UFBA      5207
666     UEFS      1952
688     UESB      1143
4503    UFRB        39
dtype: int64

In [13]:
curso_presencial.groupby(['CO_IES']).size()

CO_IES
24        972
40       1467
578      5207
599      5369
666      1952
688      1143
3984     1110
4503       39
14509    1155
dtype: int64

In [14]:
curso_presencial.loc[curso_presencial['CO_IES'] == 599, 'SG_IES'] = 'IFBA'

In [15]:
curso_presencial.loc[curso_presencial['CO_IES'] == 3984, 'SG_IES'] = 'UNIVASF'

In [16]:
curso_presencial.loc[curso_presencial['CO_IES'] == 14509, 'SG_IES'] = 'IFBAIANO'

In [28]:
curso_presencial.loc[curso_presencial['CO_IES'] == 4503, 'SG_IES'] = 'UFRB'

In [29]:
curso_presencial.groupby(['CO_IES', 'SG_IES']).size()

CO_IES  SG_IES  
24      UESC         972
40      UNEB        1467
578     UFBA        5207
599     IFBA        5369
666     UEFS        1952
688     UESB        1143
3984    UNIVASF     1110
4503    UFRB          39
14509   IFBAIANO    1155
dtype: int64

In [30]:
curso_presencial['SG_IES'].value_counts()

IFBA        5369
UFBA        5207
UEFS        1952
UNEB        1467
IFBAIANO    1155
UESB        1143
UNIVASF     1110
UESC         972
UFRB          39
Name: SG_IES, dtype: int64

In [18]:
display(curso_presencial.shape)

(18414, 108)

In [24]:
curso_presencial.groupby(['CO_CINE_ROTULO', 'NO_CURSO']).size()

CO_CINE_ROTULO  NO_CURSO                             
0114C05         CIÊNCIAS DA COMPUTAÇÃO                    533
                COMPUTAÇÃO                               2649
0612R01         REDES DE COMPUTADORES                      79
0613J01         JOGOS DIGITAIS                            112
0614C01         CIÊNCIA DA COMPUTAÇÃO                    3898
0615S02         ANÁLISE E DESENVOLVIMENTO DE SISTEMAS    2671
                SISTEMA DE INFORMAÇÃO                     477
                SISTEMAS DE INFORMAÇÃO                   3695
0616E01         ENGENHARIA DE COMPUTAÇÃO                 3062
0714E04         ENGENHARIA DA COMPUTAÇÃO                 1199
                ENGENHARIA DE COMPUTAÇÃO                   39
dtype: int64

In [22]:
curso_presencial.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18414 entries, 22527 to 2781168
Columns: 108 entries, NU_ANO_CENSO to SG_IES
dtypes: float64(100), int32(1), int64(2), object(5)
memory usage: 15.2+ MB


In [53]:
outer_merged.to_csv('dataset/alunos_curso_2015_2019_tic_br.csv', index=False)
curso_presencial.to_csv('dataset/alunos_curso_2015_2019_tic_BA.csv', index=False)

In [27]:
outer_merged['TP_CATEGORIA_ADMINISTRATIVA'].value_counts()

4.0    1218032
5.0     766828
1.0     524323
2.0     275224
3.0      15397
7.0      11168
Name: TP_CATEGORIA_ADMINISTRATIVA, dtype: int64